# Finding Similar Movies

We'll start by loading up the MovieLens dataset. Using Pandas, we can very quickly load the rows of the u.data and u.item files that we care about, and merge them together so we can work with movie names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [115]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('C:/workarea/Sample/MLDev/ml-latest-small/ratings2.csv',  delimiter= ',', header=None, names=r_cols, usecols=range(3), encoding="ISO-8859-1")
#ratings.head()

m_cols = ['movie_id', 'title']
movies = pd.read_csv('C:/workarea/Sample/MLDev/ml-latest-small/movies2.csv', dtype={'movie_id': 'Int64', 'title': str}, delimiter=',', header = None, sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")
#movies.head()

ratings = pd.merge(movies, ratings)
ratings.head()
ratings.dtypes

movie_id      int64
title        object
user_id       int64
rating      float64
dtype: object

In [116]:
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [117]:
df = pd.DataFrame(data=ratings)
writer = pd.ExcelWriter('DevMergeData.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name = 'AllMovies')
writer.save()

Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

In [118]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's extract a Series of users who rated Star Wars:

In [119]:
df = pd.DataFrame(data=movieRatings)
writer = pd.ExcelWriter('DevMoviesPivoted.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name = 'movies')
writer.save()

In [120]:
moviesToResearch = ['Black Panther (2017)', 'Toy Story (1995)', 'Jumanji (1995)', 'Avengers: Infinity War - Part I (2018)']
starWarsRatings = movieRatings[moviesToResearch[3]]
#moviesToResearch[3]
starWarsRatings.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: Avengers: Infinity War - Part I (2018), dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of Star Wars' vector of user rating with every other movie! After that, we'll drop any results that have no data, and construct a new DataFrame of movies and their correlation score (similarity) to Star Wars:

In [121]:
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

,0
title,
(500) Days of Summer (2009),-1.000000
10 Cloverfield Lane (2016),0.995871
101 Dalmatians (1996),1.000000
12 Angry Men (1957),0.219265
16 Blocks (2006),1.000000
"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",1.000000
2001: A Space Odyssey (1968),0.526830
2012 (2009),-0.500000
21 (2008),-0.500000


(That warning is safe to ignore.) Let's sort the results by similarity score, and we should have the movies most similar to Star Wars! Except... we don't. These results make no sense at all! This is why it's important to know your data - clearly we missed something important.

In [122]:
similarMovies.sort_values(ascending=False)

title
Godzilla (1998)                                                     1.0
Intouchables (2011)                                                 1.0
The Secret Life of Pets (2016)                                      1.0
Batman: The Dark Knight Returns, Part 1 (2012)                      1.0
Silver Linings Playbook (2012)                                      1.0
Minions (2015)                                                      1.0
Breakfast Club, The (1985)                                          1.0
Tomorrow Never Dies (1997)                                          1.0
Despicable Me 2 (2013)                                              1.0
Ace Ventura: When Nature Calls (1995)                               1.0
Transcendence (2014)                                                1.0
Ghostbusters (a.k.a. Ghost Busters) (1984)                          1.0
Girl with the Dragon Tattoo, The (2011)                             1.0
Transformers: Revenge of the Fallen (2009)                

Our results are probably getting messed up by movies that have only been viewed by a handful of people who also happened to like Star Wars. So we need to get rid of movies that were only watched by a few people that are producing spurious results. Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating while we're at it - that could also come in handy later.

In [123]:
import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

rating     
                                          size mean
title                                              
'71 (2014)                                 1.0  4.0
'Hellboy': The Seeds of Creation (2004)    1.0  4.0
'Round Midnight (1986)                     2.0  3.5
'Salem's Lot (2004)                        1.0  5.0
'Til There Was You (1997)                  2.0  4.0

Let's get rid of any movies rated by fewer than 100 people, and check the top-rated ones that are left:

In [124]:
popularMovies = movieStats['rating']['size'] >= 100
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                                     size      mean
title                                                              
Shawshank Redemption, The (1994)                    317.0  4.429022
Godfather, The (1972)                               192.0  4.289062
Fight Club (1999)                                   218.0  4.272936
Godfather: Part II, The (1974)                      129.0  4.259690
Departed, The (2006)                                107.0  4.252336
Goodfellas (1990)                                   126.0  4.250000
Casablanca (1942)                                   100.0  4.240000
Dark Knight, The (2008)                             149.0  4.238255
Usual Suspects, The (1995)                          204.0  4.237745
Princess Bride, The (1987)                          142.0  4.232394
Star Wars: Episode IV - A New Hope (1977)           251.0  4.231076
Schindler's List (1993)                             220.0  4.225000
Apocalypse Now (1979)                               107.0  4.219626
American History X (1998)                           129.0  4.217054
Star Wars: Episode V - The Empire Strikes Back ...  211.0  4.215640

100 might still be too low, but these results look pretty good as far as "well rated movies that people have heard of." Let's join this data with our original set of similar movies to Star Wars:

In [125]:
movieStats[popularMovies].head()

rating          
                                    size      mean
title                                             
2001: A Space Odyssey (1968)       109.0  3.894495
Ace Ventura: Pet Detective (1994)  161.0  3.040373
Aladdin (1992)                     183.0  3.792350
Alien (1979)                       146.0  3.969178
Aliens (1986)                      126.0  3.964286

In [126]:
similarMovies.head()

title
(500) Days of Summer (2009)   -1.000000
10 Cloverfield Lane (2016)     0.995871
101 Dalmatians (1996)          1.000000
12 Angry Men (1957)            0.219265
16 Blocks (2006)               1.000000
dtype: float64

In [127]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

In [128]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
2001: A Space Odyssey (1968),109.0,3.894495,0.526830
Ace Ventura: Pet Detective (1994),161.0,3.040373,0.960769
Aladdin (1992),183.0,3.792350,0.866025
Alien (1979),146.0,3.969178,0.785714
Aliens (1986),126.0,3.964286,0.645497


And, sort these new results by similarity score. That's more like it!

In [129]:
df.sort_values(['similarity'], ascending=False)[:15]

,"(rating, size)","(rating, mean)",similarity
title,,,
Ghostbusters (a.k.a. Ghost Busters) (1984),120.0,3.775000,1.000000
Dumb & Dumber (Dumb and Dumber) (1994),133.0,3.060150,1.000000
GoldenEye (1995),132.0,3.496212,1.000000
Speed (1994),171.0,3.529240,1.000000
American Pie (1999),103.0,3.378641,1.000000
"Breakfast Club, The (1985)",113.0,3.778761,1.000000
Batman Forever (1995),137.0,2.916058,1.000000
American Beauty (1999),204.0,4.056373,0.970725
Ace Ventura: Pet Detective (1994),161.0,3.040373,0.960769


Ideally we'd also filter out the movie we started from - of course Star Wars is 100% similar to itself. But otherwise these results aren't bad.

## Activity

100 was an arbitrarily chosen cutoff. Try different values - what effect does it have on the end results?